In [1]:
from prometheus_client import start_http_server, Summary, Counter, Gauge
import numpy as np

EXECUTION_TIME = Summary("function_execution_seconds", "Time spent processing.")
NEW_SYMBOLS = Counter(
    "new_symbols_fetched", "Number of new symbols fetched.", ["run_id"]
)
SYMBOLS_LENGTH = Gauge(
    "symbols_length", "Length of the list of symbols fetched.", ["run_id"]
)
TOTAL_RECORDS = Gauge(
    "total_records", "Total number of records in the profiles table.", ["run_id"]
)
FETCH_PARAMS = Gauge(
    "fetch_params",
    "Parameters passed to fetch_equity_symbols",
    ["country", "market", "run_id"],
)
MISSING_SYMBOLS = Counter(
    "missing_symbols_fetched", "Number of missing new symbols.", ["run_id"]
)

In [ ]:
from datetime import datetime

run_id = datetime.now().isoformat()


from sqlalchemy import create_engine, Column, String, Integer, Boolean, Date
from sqlalchemy.orm import declarative_base, sessionmaker
import pandas as pd
from tqdm import tqdm
import sys

pd.set_option("display.max_columns", None)

sys.path.append("..")
import os
import financedatabase as fd
from investorkit.investorkit.get_data.base import (
    get_profile,
)


from dotenv import load_dotenv

load_dotenv()


DATABASE_URL = os.getenv("DATABASE_URL")
engine = create_engine(DATABASE_URL)
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)

Base = declarative_base()


class Profile(Base):
    __tablename__ = "profiles"
    symbol = Column(String, primary_key=True, index=True)
    companyName = Column(String)
    cik = Column(Integer)
    exchange = Column(String)
    exchangeShortName = Column(String)
    industry = Column(String)
    sector = Column(String)
    country = Column(String)
    ipoDate = Column(Date)
    defaultImage = Column(Boolean)
    isEtf = Column(Boolean)
    isActivelyTrading = Column(Boolean)


Base.metadata.create_all(bind=engine)


FMP_API_KEY = os.getenv("FMP_SECRET_KEY")


@EXECUTION_TIME.time()
def fetch_equity_symbols(country="United States", market="NASDAQ Global Select"):
    equities = fd.Equities()
    selected_columns = [
        "name",
        "currency",
        "sector",
        "industry_group",
        "industry",
        "exchange",
        "market",
        "market_cap",
    ]
    us_equities = equities.select(country=country)
    df_equities = us_equities[us_equities["market"] == market][selected_columns]
    list_symbols = list(df_equities.index)

    SYMBOLS_LENGTH.labels(run_id=run_id).set(len(list_symbols))
    FETCH_PARAMS.labels(country=country, market=market, run_id=run_id).set(
        len(list_symbols)
    )

    return list_symbols


@EXECUTION_TIME.time()
def get_new_symbols(list_symbols, engine):
    existing_symbols_query = "SELECT symbol FROM profiles;"
    existing_symbols = pd.read_sql(existing_symbols_query, con=engine)
    new_symbols = list(set(list_symbols) - set(existing_symbols["symbol"].tolist()))

    NEW_SYMBOLS.labels(run_id=run_id).inc(len(new_symbols))

    return new_symbols


def store_profiles(df_profiles_filtered, engine):
    df_profiles_filtered.to_sql("profiles", con=engine, if_exists="append", index=False)


list_symbols = fetch_equity_symbols()

new_symbols = get_new_symbols(list_symbols, engine)
# new_symbols = new_symbols[70:85]


if new_symbols:
    df_profiles = get_profile(new_symbols, FMP_API_KEY)

    missing_symbols = set(new_symbols) - set(df_profiles["symbol"])
    MISSING_SYMBOLS.labels(run_id=run_id).inc(len(missing_symbols))
    list_cols = [
        "symbol",
        "companyName",
        "cik",
        "exchange",
        "exchangeShortName",
        "industry",
        "sector",
        "country",
        "ipoDate",
        "defaultImage",
        "isEtf",
        "isActivelyTrading",
    ]
    df_profiles_filtered = df_profiles[list_cols]

    df_profiles_filtered["ipoDate"].replace("", None, inplace=True)

    df_profiles_filtered["cik"].replace("", np.nan, inplace=True)

In [8]:
store_profiles(df_profiles_filtered, engine)

In [ ]:
start_http_server(8000)

In [10]:
query = "SELECT * FROM profiles;"
df = pd.read_sql(query, engine)
TOTAL_RECORDS.labels(run_id=run_id).set(len(df))